<a href="https://colab.research.google.com/github/nguyenliliana/text-classification-model-pytorch/blob/main/%5BLN%5D_v_2_Naive_Bayes_Text_Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchdata
!pip install -U torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 8.1 MB/s 
     |██████████████████████████████  | 834.1 MB 1.2 MB/s eta 0:00:44tcmalloc: large alloc 1147494400 bytes == 0x3a794000 @  0x7f8d72b54615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 0x49abe4
     |████████████████████████████████| 887.4 MB 1.7 kB/s 
     |████████████████████████████████| 140 kB 52.6 MB/s 
     |████████████████████████████████| 317.1 MB 26 kB/s 
     |████████████████████████████████| 21.0 MB 1.2 MB/s 
     |████████████████████████████████| 849 kB 72.1 MB/s 
     |████████████████████████████████| 557.1 MB 11 kB/s 
     |████████████████████████████████| 127 kB 83.0 MB/s 
  Attempting uninstall: url

# Preprocess

In [ ]:
import torch
from torchtext.datasets import AG_NEWS

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from gensim.parsing.preprocessing import remove_stopwords

import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from string import punctuation

train_iter = iter(AG_NEWS(split='train'))
tokenizer = get_tokenizer('basic_english')

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)
def clean_text(text): 
  clean = text.lower()
  clean = clean.replace(".", " ")
  clean = clean.replace("\\", " ")
  clean = clean.replace("-", " ")
  clean = clean.replace("'", " ")
  clean = clean.replace(":", " ")
  clean = clean.replace("#", " ")
  clean = clean.replace("$", " ")
  clean = clean.replace(",", " ")
  clean = clean.replace("1", " ")
  clean = clean.replace("2", " ")
  clean = clean.replace("3", " ")
  clean = clean.replace("4", " ")
  clean = clean.replace("5", " ")
  clean = clean.replace("6", " ")
  clean = clean.replace("7", " ")
  clean = clean.replace("8", " ")
  clean = clean.replace("9", " ")
  clean = clean.replace("0", " ")
  clean = clean.replace("(", " ")
  clean = clean.replace(")", " ")
  clean = strip_punctuation(clean)
  clean = remove_stopwords(clean)
  return clean
  
training_size = 84000 #70%

#BUILD VOCAB WITH TRAINING DATA
def yield_tokens(data_iter): 
  count = 0
  for label, text in data_iter: 
    if count == training_size:
      break; 
    count += 1
    yield tokenizer(clean_text(text))

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"]) #when out of vocab token is queried, return index of <unk>

print(len(vocab))
print(vocab.get_itos())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


56006
['<unk>', 's', 'new', 'said', 'reuters', 'ap', 'year', 'u', 'world', 'oil', 'company', 'monday', 'wednesday', 'tuesday', 'thursday', 'iraq', 'friday', 'york', 'yesterday', 'president', 'million', 'week', 'game', 'microsoft', 'prices', 'time', 'says', 't', 'sunday', 'united', 'corp', 'ltbgt', 'security', 'people', 'government', 'second', 'group', 'today', 'years', 'win', 'percent', 'afp', 'day', 'quot', 'quarter', 'software', 'team', 'saturday', 'open', 'internet', 'season', 'high', 'china', 'night', 'bush', 'sales', 'profit', 'market', 'record', 'minister', 'killed', 'state', 'n', 'business', 'international', 'end', 'billion', 'announced', 'home', 'stocks', 'victory', 'news', 'report', 'deal', 'washington', 'american', 'officials', 'city', 'court', 'month', 'service', 'space', 'plans', 'com', 'set', 'states', 'lead', 'british', 'technology', 'league', 'european', 'red', 'national', 'country', 'reported', 'cup', 'chief', 'talks', 'hit', 'won', 'election', 'bank', 'series', 'online

# Make Training and Testing Sets

In [ ]:
#MAKE TRAINING AND TESTING SETS
train_iter = iter(AG_NEWS(split='train'))
text_list = []
label_list = []
for label, text in train_iter:
  text_list.append(tokenizer(clean_text(text)))
  label_list.append(label)

print(len(text_list))
print(len(label_list))

train_list = text_list[0:training_size]
train_label = label_list[0:training_size]
test_list = text_list[training_size:120001]
test_label = label_list[training_size:120001]


120000
120000


# Bag of Words

In [ ]:
#MAKE BOWs

#sort training data by label

labels_set = set(train_label)

sorted_docs = {}
for i in labels_set: 
  sorted_docs[i] = []


for i in range(len(train_list)): 
    sorted_docs[train_label[i]] += train_list[i]

print(sorted_docs[1][0:20])
print(sorted_docs[2][0:20])
print(sorted_docs[3][0:20])
print(sorted_docs[4][0:20])


['venezuelans', 'vote', 'early', 'referendum', 'chavez', 'rule', 'reuters', 'reuters', 'venezuelans', 'turned', 'early', 'large', 'numbers', 'sunday', 'vote', 'historic', 'referendum', 'remove', 'left', 'wing']
['phelps', 'thorpe', 'advance', 'freestyle', 'ap', 'ap', 'michael', 'phelps', 'took', 'care', 'qualifying', 'olympic', 'meter', 'freestyle', 'semifinals', 'sunday', 'added', 'american', 'team', 'evening']
['wall', 'st', 'bears', 'claw', 'black', 'reuters', 'reuters', 'short', 'sellers', 'wall', 'street', 's', 'dwindling', 'band', 'ultra', 'cynics', 'seeing', 'green', 'carlyle', 'looks']
['madden', 'espn', 'football', 'score', 'different', 'ways', 'reuters', 'reuters', 'absenteeism', 'little', 'high', 'tuesday', 'guys', 'office', 'ea', 'sports', 'like', 'think', 'madden', 'nfl']


In [ ]:

import pandas as pd
import numpy as np
import collections

#find tf for each label

def calculateBOW(doc): 
  tf_diz = dict.fromkeys([i for i in range(len(vocab))],1)
  for word in doc: 
    tf_diz[vocab[word]] += 1; 
  return tf_diz

bows = dict.fromkeys([i for i in labels_set],{})

for i in labels_set: 
  bows[i] = calculateBOW(sorted_docs[i])

df_bow = pd.DataFrame([bows[1],bows[2], bows[3] ,bows[4]])
df_bow.head()

,0,1,2,3,4,5,6,7,8,9,...,55996,55997,55998,55999,56000,56001,56002,56003,56004,56005
0,1,12500,2545,5507,3827,3979,1091,2523,912,864,...,1,1,1,1,1,1,1,1,1,1
1,1,10477,2744,1043,1306,4121,1721,516,2310,3,...,1,2,1,1,2,1,2,1,1,2
2,1,11496,4714,5068,6129,313,2144,2298,1520,4873,...,1,1,2,1,1,1,1,2,1,1
3,1,9614,5125,2609,2318,2717,1494,1007,1426,74,...,2,1,1,2,1,2,1,1,2,1


# Naive Bayes

In [ ]:
#NAIVE BAYES
total_bow_size = 0
# for i in labels_set: 
   #total_bow_size += len(bows[i])
total_bow_size = len(train_label)
#P(label|words) = P(label) * P(word1|label) * P(word2|label) * ...
def NB(sentence):
    NB_vals = []
    for i in labels_set: 
      # p = len(bows[i])/total_bow_size
      p = train_label.count(i)/total_bow_size
      for word in sentence: 
        
        p *= ((bows[i])[vocab[word]])/len(bows[i])
      NB_vals.append(p)
    max_value = max(NB_vals)
    max_index = NB_vals.index(max_value)
    return max_index+1


# Test

In [ ]:
#TEST
def accuracy(text, label):
  correct = 0
  for i in range(len(text)):
    if(NB(text[i]) == label[i]):
      correct += 1
  return correct / len(text)
print(accuracy(test_list, test_label))

0.8961944444444444
